In [27]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [26]:
cie_10 = pd.read_excel("../data/external/CIE-10 - sin_puntos_y_X.xlsx")
regiones = (
    pd.read_excel("../data/external/Esquema_Registro-2023.xlsx", sheet_name=2, header=6)
    .dropna(how="all", axis=1)
    .iloc[:-1, :-1]
)
regiones["Código Comuna"] = regiones["Código Comuna"].astype(int)
regiones["Código Región"] = regiones["Código Región"].astype(int)

# 1. Análisis Sociodemográfico de Pacientes de Consultas

En este análisis se quiere analizar el perfil sociodemográfico de los pacientes que vienen a
realizar una consulta ambulatoria al INT. Se quieren resolver las siguientes preguntas:

- ¿Cuál es el rango etario de los pacientes atendidos?
- ¿Cuál es el sexo de los pacientes atendidos?
- ¿Cuál es la previsión de los pacientes atendidos?
- ¿Cuál es la región/comuna de los pacientes atendidos?
- ¿Cuál es el servicio de salud de los pacientes atendidos?

Además, se quiere responder cada unad e estas preguntas para cada una de las especialidades de
las consultas, diagnósticos y por tipo de consulta.


In [4]:
df_track = pd.read_csv("../data/processed/datos_limpios_track.csv", sep=";", encoding="latin-1")
df_his = pd.read_csv(
    "../data/processed/datos_limpios_diagnosticos.csv", sep=";", encoding="latin-1"
)

In [5]:
df_track["hora_completa_cita"] = pd.to_datetime(df_track["hora_completa_cita"])
df_his["fecha_reserva"] = pd.to_datetime(df_his["fecha_reserva"])

In [6]:
print(
    f"La base de Track tiene citas agendadas desde {df_track['hora_completa_cita'].min()} "
    f"a {df_track['hora_completa_cita'].max()}"
)

print(
    f"La base de HIS tiene citas agendadas desde {df_his['fecha_reserva'].min()} a "
    f"{df_his['fecha_reserva'].max()}"
)
print()
print(f"La base de Track tiene {df_track.shape[0]} citas")
print(f"La base de HIS tiene {df_his.shape[0]} citas")

La base de Track tiene citas agendadas desde 2016-01-04 08:00:00 a 2022-12-30 20:30:00
La base de HIS tiene citas agendadas desde 2016-01-04 08:00:00 a 2022-12-30 14:50:00

La base de Track tiene 758350 citas
La base de HIS tiene 265441 citas


In [7]:
datos_pacientes_unicos_por_sesion = df_track.groupby(["id_paciente", "hora_completa_cita"]).head(1)

In [8]:
df_his_y_track = pd.merge(
    df_his,
    datos_pacientes_unicos_por_sesion,
    how="left",
    left_on=["id_paciente", "fecha_reserva"],
    right_on=["id_paciente", "hora_completa_cita"],
)

sin_datos_en_track = df_his_y_track[df_his_y_track.estadocita.isna()]

In [9]:
print(
    f"Luego de unir la base de HIS con Track, la primera ha quedado con "
    f"{df_his_y_track.shape[0]} consultas (de {df_his.shape[0]}). "
    f"Hay {sin_datos_en_track.shape[0]} consultas de HIS sin información en Track"
)

Luego de unir la base de HIS con Track, la primera ha quedado con 265441 consultas (de 265441). Hay 189 consultas de HIS sin información en Track


In [10]:
for paciente in sin_datos_en_track.id_paciente.unique():
    if paciente in df_track["id_paciente"]:
        print(f"{paciente} si esta")

Los resultados anteriores indican que ninguno de los pacientes faltantes se encuentra en la base
de Track (Y no ocurre debido a que los pacientes existen en la base, pero tienen una fecha de atencion distinta a la de HIS). 
Debido a lo anterior, estos pacientes carecen de datos sociodemográficos.

Una vez realizado la unión de las bases de datos es posible realizar el conteo anidado por
las variables sociodemográficas.

In [11]:
VARIABLES_SOCIODEMOGRAFICAS_INTERES = [
    "ano",
    "sexo_x",
    "comuna",
    "provincia",
    "region",
    "plan",
    "prevision",
    "tipoatencion",
    "rango_etario",
]

VARIABLE_A_CONTAR = "codigo_diagnostico"

In [12]:
resultado_sociodemografico = (
    df_his_y_track.groupby(VARIABLES_SOCIODEMOGRAFICAS_INTERES)[VARIABLE_A_CONTAR]
    .value_counts()
    .reset_index(name="cantidad_de_consultas")
)

In [13]:
resultado_sociodemografico.to_excel(
    "../data/interim/sociodemografico/desglose_sociodemografico.xlsx", index=False
)

PermissionError: [Errno 13] Permission denied: '../data/interim/sociodemografico/desglose_sociodemografico.xlsx'